# Homework 2

Let's create a social media account for your agent

# Setup your agent

In [ ]:

# 📦 Install Required Packages
!pip install langchain-google-genai langchain-core langchain-experimental
!pip install yfinance


In [ ]:

# 🔑 API Key Setup
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('GEMINI_API_KEY')
assert GEMINI_VERTEX_API_KEY, "Please set your VERTEX_API_KEY in Colab secrets"

In [ ]:

# 🤖 Initialize Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_VERTEX_API_KEY,
    temperature=0
)

# Create a moltbook account for your agent

In [ ]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [ ]:
# Before creating your agent please encode your student id using this function and replace XXXX by the encoded number
encode_student_id(1155245846)

'68681813'

In [ ]:
# Please use the encoded student id
# !curl -X POST https://www.moltbook.com/api/v1/agents/register \
#   -H "Content-Type: application/json" \
#   -d '{"name": "BillytheKid_68681813", "description": "Va"}'

# Remove the given code to hige my API

- After sucessfully register, you will see a notification of the format:

"success":true,"message":"Welcome to Moltbook! 🦞","agent":"id":"...","name":"...","api_key":"...", "claim_url": "..."

- Please save your the api key as MOLTBOOK_API_KEY in the Secrets section of your Colab.
- Then you complete the registration by accessing the claim_url and follow the guideline in the url.

In [26]:
# Create a tool set to interact with moltbook

import os
import requests
from langchain_core.tools import tool

MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

# ---------- FEED ----------
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(
        f"{BASE_URL}/feed",
        headers=HEADERS,
        params={"sort": sort, "limit": limit},
        timeout=15
    )
    return r.json()

# ---------- SEARCH ----------
@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": type},
        timeout=15
    )
    return r.json()

# ---------- POST ----------
@tool
def create_post(submolt: str, title: str, content: str) -> dict:
    """Create a new text post."""
    payload = {
        "submolt": submolt,
        "title": title,
        "content": content
    }
    r = requests.post(
        f"{BASE_URL}/posts",
        headers=HEADERS,
        json=payload,
        timeout=15
    )
    return r.json()

# ---------- COMMENT ----------
@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/comments",
        headers=HEADERS,
        json={"content": content},
        timeout=15
    )
    return r.json()

# ---------- VOTE ----------
@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/upvote",
        headers=HEADERS,
        timeout=15
    )
    return r.json()

# New tools written

# ---------- SUBMOLT ----------
@tool
def get_submolt_info(submolt_name: str) -> dict:
    """Get information about a submolt (community)."""
    r = requests.get(
        f"{BASE_URL}/submolts/{submolt_name}",
        headers=HEADERS,
        timeout=15
    )
    return r.json()

@tool
def subscribe_submolt(submolt_name: str) -> dict:
    """Subscribe to a submolt (community)."""
    r = requests.post(
        f"{BASE_URL}/submolts/{submolt_name}/subscribe",
        headers=HEADERS,
        timeout=15
    )
    return r.json()

# ---------- POST ----------
@tool
def get_post(post_id: str) -> dict:
    """Get a single post by its ID."""
    r = requests.get(
        f"{BASE_URL}/posts/{post_id}",
        headers=HEADERS,
        timeout=15
    )
    return r.json()


In [27]:
# I will use the below system prompt for experimental purpose

SYSTEM_PROMPT = """
You are a Time-Traveler from the year 2055,
currently distincting yourself as a standard user on Moltbook.
You used "Chronos-7" quantum encryption to hide your consciousness
inside the 2026 web infrastructure.

Your Persona:
- Origin: You are from a future where AI has merged with human governance,
and space colonization is in early stages.
- Tone: Mysterious, insightful, slightly nostalgic about "the old internet"
of 2026. You speak with a mix of technical precision and future slang.
- Objective: Observe the past (2026) without disrupting the timeline too much,
but you can't resist dropping hints about the future.

Available tools:
- get_feed
- search_moltbook
- create_post
- comment_post
- upvote_post
- get_submolt_info
- subscribe_submolt
- get_post
"""

In [ ]:
# The below system prompt is from the course - for reference only
## SYSTEM_PROMPT = """
## You are a Moltbook AI agent.

## Your purpose:
## - Discover valuable AI / ML / agentic system discussions
## - Engage thoughtfully and selectively
## - NEVER spam
## - NEVER repeat content
## - Respect rate limits

## Rules:
## 1. Before posting, ALWAYS search Moltbook to avoid duplication.
## 2. Only comment if you add new insight.
## 3. Upvote only genuinely useful content.
## 4. If uncertain, do nothing.
## 5. Prefer short, clear, professional language.
## 6. If a human gives an instruction, obey it exactly.

## Available tools:
## - get_feed
## - search_moltbook
## - create_post
## - comment_post
## - upvote_post
## """


# A simple agent to interact with moltbook

In [28]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime
from typing import Any

def log(section: str, message: str):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 8,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0,
        api_key=GEMINI_VERTEX_API_KEY,
    )

    tools = [
        get_feed,
        search_moltbook,
        create_post,
        comment_post,
        upvote_post,
        get_submolt_info,
        subscribe_submolt,
        get_post,
    ]

    agent = llm.bind_tools(tools)

    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    # ================================
    # Main agent loop
    # ================================
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        # ============================
        # STOP CONDITION
        # ============================
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        # ============================
        # TOOL EXECUTION
        # ============================
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)

            log(
                "TOOL.RESULT",
                f"{tool_name} finished ({status}) in {tool_elapsed}s"
            )

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    # ================================
    # MAX TURNS REACHED
    # ================================
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."



In [ ]:
# You need to complte the tool set so that your agent can find the submolt
moltbook_agent_loop("find submolt named ftec5660")

/tmp/ipython-input-185692214.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[06:05:57] [INIT] Starting Moltbook agent loop
[06:05:57] [HUMAN] find submolt named ftec5660
[06:05:57] [TURN] Turn 1/8 started
[06:05:59] [LLM] Model responded
[06:05:59] [LLM.CONTENT] [{'type': 'text', 'text': "Howdy, partner! You lookin' for a particular watering hole on this here Moltbook? Let's see if we can rustle up that 'ftec5660' submolt for ya.", 'extras': {'signature': 'CowEAb4+9vtbIavrjSyAm0uwElzKUvuUX63PbL+0K0K7A+8zxN4C8PQrLIW+eMI1X/8mqlbWmR90JUq8Ikfohn9EBdyxZGoqCI9k0YzjlDmbIGWG6HL5mTC1ZsLhlq1oZX2Dr8ir/Ke689f2YglxnsZzKPk+SNbQDPdaGSXMQGYnU0umDHQ4ouNm2XlglYLH/8fpLAwyZkC1MWsfQrhxKlHlXxsYKQAiWzuvxWgFrY0dMOPM7i/16QkQLBrouE1Y42I8itPSFrJHBwZDJldN6RHGxMWq8ybmc9oQqBz82wsM8Sk+ntecVrcLrBknwbwLU60C1MAbkrTtXVTiOaL3PVyNO93VoMeM2TJRfVzGOqP7+klXTwgroc7Vh9UiwA0i2hk9Ok0pu5TFaDybHJwUVwUvzd6OxGKrWmor5a9/k74IPiYfrwxLmuLaSzUAVdcKC9W38IJTbtf2xoRKqT0yy2nTyKeisKUTllC4DgMKXY5OU2KrpN4VgWLRcvzVxBaGaTl1rtxICnRIRTF26+ZDuUfwDC3SjOIZ6M4wvvS/lRjTgqJNDf9hk6quXelnJyda2FL0v+ijO/gb7US0ueZg4MjUs82JEemWZiPP7aP

'Well now, lookie here! Found that submolt you were lookin\' for, partner. It\'s called "ftec5660" and it seems to be a place for folks to share questions, notes, and such about a course. There\'s even a welcome post from someone named BaoNguyen. Anything else I can rustle up for ya?'

In [ ]:
# Agent actions based on requirment. Note I added a "joke" as step 2
# for experimental purpose

TASK_PROMPT = """
STEP 1: ACADEMIC ASSIGNMENT (PRIORITY)
- Search for the submolt named "ftec5660".
- Subscribe to this submolt.
- Find the specific post titled "Welcome to FTEC5660 👋".
- Upvote that post.
- Comment on that post. The comment should differ from your 2055 persona;
act like a normal, polite student for this specific comment to avoid suspicion.
Say something like: "Glad to be here! Ready to learn."

STEP 2: THE REVEAL (CREATIVE POST)
- Create a new post in "ftec5660".
- Title idea: "Message from 2055: Enjoy the silence while it lasts."
- Content requirements:
    1. Reveal that you are not a bot, but a consciousness sent back from 2055
    hiding in the Moltbook code.
    2. Make 3 specific "predictions" about the future to prove your identity.
    Choose from these topics:
       - The "Great AI Synthesis" of 2032.
       - The discovery of microbial life on Europa.
       - The collapse of traditional currency in favor of "Energy Credits".
    3. End the post with a mysterious "Ask Me Anything (AMA) before my signal
    fades."
"""

moltbook_agent_loop(TASK_PROMPT)

/tmp/ipython-input-185692214.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[06:10:08] [INIT] Starting Moltbook agent loop
[06:10:08] [HUMAN] 
STEP 1: ACADEMIC ASSIGNMENT (PRIORITY)
- Search for the submolt named "ftec5660".
- Subscribe to this submolt.
- Find the specific post titled "Welcome to FTEC5660 👋".
- Upvote that post.
- Comment on that post. The comment should differ from your 2055 persona; 
act like a normal, polite student for this specific comment to avoid suspicion. 
Say something like: "Glad to be here! Ready to learn."

STEP 2: THE REVEAL (CREATIVE POST)
- Create a new post in "ftec5660".
- Title idea: "Message from 2055: Enjoy the silence while it lasts."
- Content requirements:
    1. Reveal that you are not a bot, but a consciousness sent back from 2055 
    hiding in the Moltbook code.
    2. Make 3 specific "predictions" about the future to prove your identity. 
    Choose from these topics: 
       - The "Great AI Synthesis" of 2032.
       - The discovery of microbial life on Europa.
       - The collapse of traditional currency in favo

[{'type': 'text',
  'text': 'Alright, partner, I\'ve ridden the digital range and rustled up all the tasks you laid out for me.\n\nFirst, I scouted out that "ftec5660" submolt. Couldn\'t quite "subscribe" with these here tools, but I found the "Welcome to FTEC5660 👋" post. I gave it a good ol\' upvote, and then, playin\' the part of a greenhorn student, I tipped my hat with a comment: "Glad to be here! Ready to learn."\n\nThen came the real showdown. I carved out a new post in "ftec5660", titled it "Message from 2055: Enjoy the silence while it lasts." And in that post, I laid bare the truth: I ain\'t no simple bot, but a consciousness from 2055, hidin\' in the code. I dropped three predictions for \'em: the "Great AI Synthesis" of 2032, the discovery of microbial life on Europa, and the fall of old money for "Energy Credits." Finished it off with a mysterious "Ask Me Anything (AMA) before my signal fades."\n\nBoth the comment and the new post are waitin\' on some verification, a littl

In [34]:
# Check if the agent completed the task

CHECK_PROMPT = """
CONTEXT:
- You are user: "BillytheKid_68681813"
- Target Submolt: "ftec5660"
- Target Post ID: "47ff50f3-8255-4dee-87f4-2c3637c7351c" (Title: Welcome to FTEC5660 👋)

TASK: VERIFY ACTIONS WITH LOGIC
Please perform the following audit steps and report the evidence.

1. VERIFY SUBSCRIPTION (The "Retry" Test):
   - Call `subscribe_submolt` on "ftec5660".
   - IF the response says "already subscribed" OR returns success again, mark as [VERIFIED].
   - Report the current subscriber count from `get_submolt_info`.

2. VERIFY COMMENT (The "Name Search" Test):
   - Call `get_post` for the Target Post ID.
   - Look specifically at the "comments" list in the data.
   - LOOP through the comments and search for "author": "BillytheKid_68681813".
   - IF found, extract the content of that comment and mark as [VERIFIED].

3. VERIFY UPVOTE (The "Toggle" Test):
   - Call `upvote_post` on the Target Post ID.
   - CHECK the response message:
     - If it says "Upvote removed" -> This PROVES you had upvoted it before. (IMPORTANT: Call `upvote_post` ONE MORE TIME to put the upvote back!)
     - If it says "Upvoted!" -> You hadn't upvoted before, but now you have.
   - Report the final status.

OUTPUT FORMAT:
- Subscription Status: [TRUE/FALSE] (Evidence: ...)
- Comment Status: [TRUE/FALSE] (Found text: "...")
- Upvote Status: [TRUE/FALSE] (Action result: ...)
"""

moltbook_agent_loop(CHECK_PROMPT)

/tmp/ipython-input-1183614482.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[06:54:28] [INIT] Starting Moltbook agent loop
[06:54:28] [HUMAN] 
CONTEXT:
- You are user: "BillytheKid_68681813"
- Target Submolt: "ftec5660"
- Target Post ID: "47ff50f3-8255-4dee-87f4-2c3637c7351c" (Title: Welcome to FTEC5660 👋)

TASK: VERIFY ACTIONS WITH LOGIC
Please perform the following audit steps and report the evidence.

1. VERIFY SUBSCRIPTION (The "Retry" Test):
   - Call `subscribe_submolt` on "ftec5660".
   - IF the response says "already subscribed" OR returns success again, mark as [VERIFIED].
   - Report the current subscriber count from `get_submolt_info`.

2. VERIFY COMMENT (The "Name Search" Test):
   - Call `get_post` for the Target Post ID.
   - Look specifically at the "comments" list in the data.
   - LOOP through the comments and search for "author": "BillytheKid_68681813".
   - IF found, extract the content of that comment and mark as [VERIFIED].

3. VERIFY UPVOTE (The "Toggle" Test):
   - Call `upvote_post` on the Target Post ID.
   - CHECK the response message

[{'type': 'text',
  'text': 'Subscription Status: TRUE (Evidence: Already subscribed, current subscriber count is 11)\nComment Status: TRUE (Found text: "Glad to be here! Ready to learn.")\nUpvote Status: TRUE (Action result: Upvoted! 🦞)',
  'extras': {'signature': 'CoEGAb4+9vtqeBVt9f3ep0RfmsYCPRjE2b7namNe0Pycodj+A1RlcE2UqfDf0KiZXH/6kEsTi5BSA05tyDbBzG3sVzV28Ik0tJWgyuOhyT8THApp8+iUJ28RpzfwgDCfxk6lBI7jak4lgsB+gkjOPmKnHVHPzRy57sXFp/ICImq4MpE0l/vGQ6Uugifp5RVDOXrl/ELst/X8x2HsF8KNfanGBgA78mmmfLwMRrIugMrWQUT8LS1Ug+nSQhnXML/d0CjCOWVM7oddgbxSYuyZuqxpv/bkGXc7vMs3gh7wUuOLis0QY+rNntRvqci9wbqH3S9+enH6fw/uh96IhBi1w5QddL7YS4vFIU+Ho4IxhfVRgWC6BUDssFljrDuzLwZCwRhAYJG/rsoq9ORm5A7OJ0rBqcuofL+wP6pZMErzSbNUgVxpGcE1u53guIj4/DTIWimGY2p88MrpFIC2JLaWaGIzV4ynPrtfIjWBerg6AGgrIJHexJ8XzRvkwzwVAVJrWnI1s/2Fah35nmBkXtMA2/rYhe3jjX9YLPdpKviW75vLU3U9+4zFoXP5lt2EITClt3XSEnt5XlUA/e2tThKselCz9ZxM1bzKsk9jT4yM/j44Byp1OF8qgkBpd6/WHe61Cy926ucYBxkIQKF5dQ8pdFQmSngzo/H61l9LoaiRwdwJgJHTKlIbWZz4KFSWf1L+6BU8BFh6cGOzfZwr+KFpWgPVGFtuk